In [1]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
# os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
# import torch
# print('Current cuda device:', torch.cuda.current_device())
# print('Count of using GPUs:', torch.cuda.device_count())

In [2]:
import os
import random
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import ElectraModel, ElectraTokenizer, ElectraConfig

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [3]:
STATE_DICT_PATH = 'training_results/v1.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

labels = ['[PAD]', 'E_B', 'E_I', 'O']
num_labels = len(labels)
id2label = {k: v for k, v in enumerate(labels)}
label2id = {v: k for k, v in id2label.items()}

max_len = 256
batch_size = 32

## Load Model and Tokenizer

In [4]:
config = ElectraConfig.from_pretrained('config')
bert = ElectraModel(config)
tokenizer = ElectraTokenizer.from_pretrained('tokenizer')

In [5]:
class BERTPoSTagger(nn.Module):
    def __init__(self, bert, output_dim, dropout):
        super().__init__()
        
        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text, mask):
        embedded = self.dropout(self.bert(text, mask)[0])
        predictions = self.fc(self.dropout(embedded))
        return predictions

In [6]:
OUTPUT_DIM = num_labels
DROPOUT = 0.25

model = BERTPoSTagger(bert, OUTPUT_DIM, DROPOUT)
model.bert.resize_token_embeddings(len(tokenizer))

NGPU = torch.cuda.device_count()
if NGPU > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))

model.to(device)
model.load_state_dict(torch.load(STATE_DICT_PATH))

<All keys matched successfully>

## Load Data

In [7]:
data = pd.read_pickle('data/preprocessed.pkl')

data['tokens'] = data.tokens.apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
data['labels'] = data.labels.apply(lambda x: ['O'] + x + ['O'])

data['tokens'] = data.tokens.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))
data['labels'] = data.labels.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))

tokens_lst = data.tokens.to_list()
labels_lst = data.labels.to_list()

X_train, X_eval, y_train, y_eval = train_test_split(tokens_lst, 
                                                    labels_lst, 
                                                    test_size=0.2, shuffle=True, random_state=42)

train_data = []
for tokens, labels in zip(X_train, y_train):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)

    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    train_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])
    
eval_data = []
for tokens, labels in zip(X_eval, y_eval):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)
    
    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    eval_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])
    
class TaggerDataset(Dataset): 
    def __init__(self, data):
        self.data = data
    
    def __len__(self): 
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = self.data[idx][0]
        mask = self.data[idx][1]
        label_ids = self.data[idx][2]
        return (torch.LongTensor(input_ids), torch.LongTensor(mask), torch.LongTensor(label_ids))
    
train_dataset = TaggerDataset(train_data)
eval_dataset = TaggerDataset(eval_data)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
eval_loader = DataLoader(eval_dataset, batch_size = batch_size, shuffle = True)

## Functions

In [8]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])

    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [9]:
def categorical_f1(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    max_preds_no_pad = max_preds[non_pad_elements].squeeze(1).detach().cpu()
    y_no_pad = y[non_pad_elements].detach().cpu()
    
    f1_macro = f1_score(y_no_pad, max_preds_no_pad, average='macro')
    f1_micro = f1_score(y_no_pad, max_preds_no_pad, average='micro')    
    
    return f1_macro, f1_micro

In [10]:
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [11]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    predictions_set = None
    tags_set = None
    with torch.no_grad():
        for batch in iterator:
            text = batch[0].to(device)
            mask = batch[1].to(device)
            tags = batch[2].to(device)
            
            predictions = model(text, mask)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            if predictions_set == None:
                predictions_set = predictions
                tags_set = tags
            else:
                predictions_set = torch.cat([predictions_set, predictions], dim=0)
                tags_set = torch.cat([tags_set, tags], dim=0)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        f1_macro, f1_micro = categorical_f1(predictions_set, tags_set, tag_pad_idx)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1_macro, f1_micro

## Inferrence

In [12]:
for batch in eval_loader:
    input_ids, mask, label_ids = batch
    input_ids, mask, label_ids = input_ids.to(device), mask.to(device), label_ids.to(device)
    
    output = model(input_ids, mask).argmax(-1)
    break

In [13]:
preds = output.tolist()
preds = {'preds': preds}
preds = pd.DataFrame(preds)

In [14]:
preds2labels = preds.preds.apply(lambda lst: list(map(lambda x: id2label[x], lst)))

In [15]:
inputs = input_ids.tolist()
inputs = {'inputs': inputs}
inputs = pd.DataFrame(inputs)

In [16]:
inputs2labels = inputs.inputs.apply(lambda lst: list(map(lambda x: tokenizer.convert_ids_to_tokens(x), lst)))

In [20]:
for ins, outs in zip(inputs2labels, preds2labels):
    idx = ins.index('[PAD]')
    ins, outs = ins[:idx], outs[:idx]
    for x, y in zip(ins, outs):
        print(x, y)
    print()

[CLS] O
충청북도 O
음성군 O
소재 O
제조업 O
공장 O
내 O
##에 O
##서 O
재 O
료 O
혼합기 O
에 O
땅콩 O
##피 O
##를 O
투입 O
하 O
##기 O
위해 O
발판 E_B
에 E_I
올라가 E_I
작업 E_I
중 O
몸 O
##의 O
중 E_B
심 E_I
##을 E_I
잃 E_I
##고 E_I
콘크리트 O
바닥 O
##으로 O
떨어 E_B
##짐 E_I
. O
[SEP] O

[CLS] O
2020 O
##년 O
7 O
##월 O
12 O
##일 O
오전 O
9 O
##시 O
미 O
해 O
군 O
기지 O
##에 O
##서 O
선박 O
에서 O
폭발 E_I
##하 O
##고 O
##가 O
발생 O
##하여 O
21 O
##명 O
##이 O
부상 E_B
##을 O
입 O
##는 O
사고 O
##가 O
발생 O
##하 O
##였 O
##다 O
. O
부상자 O
##는 O
민간인 O
4 O
명과 O
선원 O
17 O
##명 O
##이 O
부상 E_B
##을 O
입고 O
병원 O
##으로 O
이송 O
##되 O
##었 O
##으며 O
생명 E_B
##에 O
##는 O
지장 E_I
##이 E_I
없 E_I
##는 E_I
것 E_I
##으로 O
알려 O
##지 O
##고 O
있 O
##다 O
. O
해당 O
선박 O
은 O
8 O
4 O
0 O
##ft O
규모 O
##이 O
##며 O
폭발 O
##이 O
시작 O
##된 O
지점 O
##은 O
정확히 O
알 O
수 O
없 O
##다 O
. O
사고 O
당시 O
해당 O
선박 O
은 O
일상 O
##적 O
##인 O
정비 O
작업 O
을 O
수행 O
중 O
이 O
##었 O
##다 O
. O
[SEP] O

[CLS] O
2020 O
##년 O
2 O
##월 O
13 O
##일 O
새 O
벽 O
4 O
시 O
30 O
##분 O
##경 O
미국 O
뉴 O
##버리 O
포트 O
의 O
한 O
화학 O
물질 O
제조공장 O
에서 O
위험 O
##물 O
##질 E_I
폭발 E_I
##사고 O
